In [1]:
from network import DSGR
import dgl
from dgl import load_graphs
import torch
import numpy as np

c:\Users\Fastora\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Fastora\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Fastora\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
graph_path = './data/recipes50_graph'
graph = load_graphs(graph_path)[0][0]

In [3]:
import pandas as pd

df = pd.read_csv('./data/recipes50.csv')


#get user with user_id = 14326
user_id = 14326
user = df[df['user_id'] == user_id]


In [4]:

user_data = user.sort_values(['time'], kind='mergesort')
print('Number of recipes: ', len(user_data))

#print the items_id
print('Items: ', user_data['item_id'].values)


Number of recipes:  3
Items:  [4942 2293 9323]


In [5]:
test_user = 14326
test_items = user_data['item_id'].values
test_times = user_data['time'].values


graphs = []
for j, t in enumerate(test_times[0:-1]):
    if j == 0:
        continue
    if j < 50:
        start_t = test_times[0]
    else:
        start_t = test_times[j - 50]
    sub_u_eid = (graph.edges['by'].data['time'] < test_times[j + 1]) & (graph.edges['by'].data['time'] >= start_t)
    sub_i_eid = (graph.edges['pby'].data['time'] < test_times[j + 1]) & (graph.edges['pby'].data['time'] >= start_t)
    sub_graph = dgl.edge_subgraph(graph, edges={'by': sub_u_eid, 'pby': sub_i_eid}, relabel_nodes=False)
    u_temp = torch.tensor([test_user])
    his_user = torch.tensor([test_user])
    graph_i =  dgl.sampling.select_topk(sub_graph,50, weight='time', nodes={'user': u_temp})
    i_temp = torch.unique(graph_i.edges(etype='by')[0])
    his_item = torch.unique(graph_i.edges(etype='by')[0])
    edge_i = [graph_i.edges['by'].data[dgl.NID]]
    edge_u = []
    for _ in range(3-1):
        graph_u = dgl.sampling.select_topk(sub_graph, 50, weight='time', nodes={'item': i_temp})  # item的邻居user
        u_temp = np.setdiff1d(torch.unique(graph_u.edges(etype='pby')[0]), his_user)[-50:]
        #u_temp = torch.unique(torch.cat((u_temp, graph_u.edges(etype='pby')[0])))
        graph_i = dgl.sampling.select_topk(sub_graph, 50, weight='time', nodes={'user': u_temp})
        his_user = torch.unique(torch.cat([torch.tensor(u_temp), his_user]))
        #i_temp = torch.unique(torch.cat((i_temp, graph_i.edges(etype='by')[0])))
        i_temp = np.setdiff1d(torch.unique(graph_i.edges(etype='by')[0]), his_item)
        his_item = torch.unique(torch.cat([torch.tensor(i_temp), his_item]))
        edge_i.append(graph_i.edges['by'].data[dgl.NID])
        edge_u.append(graph_u.edges['pby'].data[dgl.NID])
    all_edge_u = torch.unique(torch.cat(edge_u))
    all_edge_i = torch.unique(torch.cat(edge_i))
    fin_graph = dgl.edge_subgraph(sub_graph, edges={'by':all_edge_i,'pby':all_edge_u})
    target = test_items[j+1]
    last_item = test_items[j]
    u_alis = torch.where(fin_graph.nodes['user'].data['user_id']==test_user)[0]
    last_alis = torch.where(fin_graph.nodes['item'].data['item_id']==last_item)[0]
    user = test_user
    user = torch.tensor([user])
    target = torch.tensor([target])
    graphs.append((fin_graph, u_alis, last_alis, target, user))

asd = 5


In [6]:
print(graphs)

[(Graph(num_nodes={'item': 269, 'user': 868},
      num_edges={('item', 'by', 'user'): 371, ('user', 'pby', 'item'): 1244},
      metagraph=[('item', 'user', 'by'), ('user', 'item', 'pby')]), tensor([32]), tensor([45]), tensor([9323]), tensor([14326]))]


In [7]:
# model = DGSR(user_num=89024, item_num=17676, input_dim=50, item_max_length=50,
#              user_max_length=50, feat_drop=0.3, attn_drop=0.3, user_long='orgat', user_short='att',
#              item_long='orgat', item_short='att', user_update='rnn', item_update='rnn', last_item=False,
#              layer_num=2).cuda()

model = DSGR(89024, 17676, 50, feat_drop=0.3, attn_drop=0.3).cuda()


In [8]:
mapping = {
    "user_embedding.weight": "user_emb.weight",
    "item_embedding.weight": "recipe_emb.weight",
    "layers.0.agg_gate_u.weight": "dsgr_layers.0.agg_gate_user.weight",
    "layers.0.agg_gate_i.weight": "dsgr_layers.0.agg_gate_recipe.weight",
    "layers.0.user_weight.weight": "dsgr_layers.0.user_weight.weight",
    "layers.0.item_weight.weight": "dsgr_layers.0.recipe_weight.weight",
    "layers.0.user_update.weight": "dsgr_layers.0.user_update.weight",
    "layers.0.item_update.weight": "dsgr_layers.0.recipe_update.weight",
    "layers.0.last_weight_u.weight": "dsgr_layers.0.user_last_weight.weight",
    "layers.0.last_weight_i.weight": "dsgr_layers.0.recipe_last_weight.weight",
    "layers.0.i_time_encoding.weight": "dsgr_layers.0.recipe_date_emb.weight",
    "layers.0.i_time_encoding_k.weight": "dsgr_layers.0.recipe_date_emb_k.weight",
    "layers.0.u_time_encoding.weight": "dsgr_layers.0.user_date_emb.weight",
    "layers.0.u_time_encoding_k.weight": "dsgr_layers.0.user_date_emb_k.weight",
    "layers.1.agg_gate_u.weight": "dsgr_layers.1.agg_gate_user.weight",
    "layers.1.agg_gate_i.weight": "dsgr_layers.1.agg_gate_recipe.weight",
    "layers.1.user_weight.weight": "dsgr_layers.1.user_weight.weight",
    "layers.1.item_weight.weight": "dsgr_layers.1.recipe_weight.weight",
    "layers.1.user_update.weight": "dsgr_layers.1.user_update.weight",
    "layers.1.item_update.weight": "dsgr_layers.1.recipe_update.weight",
    "layers.1.last_weight_u.weight": "dsgr_layers.1.user_last_weight.weight",
    "layers.1.last_weight_i.weight": "dsgr_layers.1.recipe_last_weight.weight",
    "layers.1.i_time_encoding.weight": "dsgr_layers.1.recipe_date_emb.weight",
    "layers.1.i_time_encoding_k.weight": "dsgr_layers.1.recipe_date_emb_k.weight",
    "layers.1.u_time_encoding.weight": "dsgr_layers.1.user_date_emb.weight",
    "layers.1.u_time_encoding_k.weight": "dsgr_layers.1.user_date_emb_k.weight",
    "unified_map.weight": "unified_map.weight"
}


In [9]:
saved_state_dict = torch.load("./model_8.pt")

# Create a new state_dict for the model you want to load
new_state_dict = {}

# Update the state_dict keys using the mapping
for key, value in saved_state_dict.items():
    if key in mapping:
        new_state_dict[mapping[key]] = value

# Load the updated state_dict into your model
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [10]:
graphs_gpu = []
for i in range(len(graphs)):
    graphs_gpu.append((graphs[i][0].to('cuda'), graphs[i][1].to('cuda'), graphs[i][2].to('cuda'), graphs[i][3].to('cuda'), graphs[i][4].to('cuda')))

In [11]:
graph = graphs_gpu[0][0]
user = graphs_gpu[0][1]
last_item = graphs_gpu[0][2]

print(graph)
print(user)
print(last_item)

Graph(num_nodes={'item': 269, 'user': 868},
      num_edges={('item', 'by', 'user'): 371, ('user', 'pby', 'item'): 1244},
      metagraph=[('item', 'user', 'by'), ('user', 'item', 'pby')])
tensor([32], device='cuda:0')
tensor([45], device='cuda:0')


In [12]:
#score = model(graph, user, last_alis, is_training=True)

scores = []
for i in range(len(graphs_gpu)):
    score = model(graphs_gpu[i][0], graphs_gpu[i][1], graphs_gpu[i][2], training=True)
    scores.append(score)

In [13]:
#for each score in scores, find the top 10 items then get the average of the top 10 items
top_10 = []
for i in range(len(scores)):
    top_10.append(torch.topk(scores[i], 10)[1])
    
    
#detaching the tensor from gpu and converting it to numpy
top_10 = [i.cpu().detach().numpy() for i in top_10]

top_dict = {}

for i in range(len(top_10)):
    for j in range(len(top_10[i][0])):
        if top_10[i][0][j] not in top_dict:
            top_dict[top_10[i][0][j]] = 1
        else:
            top_dict[top_10[i][0][j]] += 1



In [14]:
#sort the dictionary by value
sorted_dict = sorted(top_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_dict)

[(3490, 1), (2387, 1), (6904, 1), (2293, 1), (3006, 1), (4787, 1), (6018, 1), (11243, 1), (11218, 1), (10150, 1)]


In [15]:
#squeeze the score
score = score.squeeze()

#print top 10 scores
print(score.topk(10))

#get the top 10 items
top_10 = torch.topk(score, 10)




torch.return_types.topk(
values=tensor([4.2394, 4.1812, 4.0032, 3.9885, 3.7544, 3.7251, 3.5824, 3.5659, 3.5402,
        3.5073], device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([ 3490,  2387,  6904,  2293,  3006,  4787,  6018, 11243, 11218, 10150],
       device='cuda:0'))


In [16]:
print('Top 10 items: ', top_10.indices.cpu().numpy())


Top 10 items:  [ 3490  2387  6904  2293  3006  4787  6018 11243 11218 10150]
